In [21]:
from dotenv import load_dotenv
import os
import base64
import requests
import pandas as pd
import numpy as np

In [22]:
# Get configuration file that has all the api tokens
def configure():
    load_dotenv()
    CLIENT_ID = os.getenv('CLIENT_ID')
    CLIENT_SECRET = os.getenv('CLIENT_SECRET')
    return CLIENT_ID, CLIENT_SECRET

In [23]:
#Grab the Client_id and Client_secret from the env for the documented api
CLIENT_ID, CLIENT_SECRET = configure()
print(CLIENT_ID)
print(CLIENT_SECRET)

40692bc74b7f42f9a0d19365e7af4cd8
790e90e05ada42339bc081c69ffef37e


***

In [24]:
def get_token(CLIENT_ID,CLIENT_SECRET):
    '''Takes the Client_ID and Client_Secret to request access token.
        Access tokens expire every hour so have to request new one'''
    ## Setup the authorization str and convert to base64  
    auth_str = CLIENT_ID + ":" + CLIENT_SECRET
    #encode str with utf-8 first
    auth_bytes = auth_str.encode("utf-8")
    #encoding it to required base64
    auth_base64 = str(base64.b64encode(auth_bytes),"utf-8")

    ## Run the Actual Request with Post, setting up required header fields
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}

    ## actually make the request to Spotify
    result = requests.post(url, headers = headers, data = data)
    json_result = result.json()
    token = json_result["access_token"]
    return token

In [25]:
def get_auth_header(token):
    '''Creates Authorization header with the access token for requests to official Spotify API'''
    header = {
        "Authorization": "Bearer " + token
        }
    return header

In [26]:
#get Token
token = get_token(CLIENT_ID,CLIENT_SECRET)

In [27]:
token

'BQC1tzFb_5-VN3fxRXPD8HHsy03Mm0GTbw7kck4jD_qNzYoG-OVKYjTfvDGixQJkYVw_W24Dy3xBoeyTexgxgtATIP9enk3jpaU8BlINMPKYEXMxhu6b'

***

## Search for artist_id by artist_name

In [28]:
def search_artist(token, artist_name):
    '''Search the Artist name and get their info including ID and popularity'''
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)

    querystring = {
                "q": artist_name,
                "type": "artist",
                "limit": 1
                }

    result = requests.request("GET",url, headers= headers, params=querystring)
    
    json_result = result.json()

    try:
        #returns a artist_id, artist name,
        answer = json_result['artists']['items'][0]
        artist_info = {'artist_name':answer['name'],'artist_id':answer['id'], 'artist_followers':answer['followers']['total'], 'artist_popularity':answer['popularity'], 'artist_genres':answer['genres']}
        return artist_info
    except:
        #give error
        print("No artist with this name on Spotify")
        return None
    

Test what `search_artist()` return

In [29]:
artist = search_artist(token, "Niki")

In [30]:
artist

{'artist_name': 'NIKI',
 'artist_id': '2kxP07DLgs4xlWz8YHlvfh',
 'artist_followers': 2360619,
 'artist_popularity': 76,
 'artist_genres': ['indonesian r&b', 'pop']}

In [31]:
artist['artist_id']

'2kxP07DLgs4xlWz8YHlvfh'

 ## Search for albums and singles by artist_id

#### `get_singles_albums_by_artist_limit_50()`

In [32]:
def get_singles_albums_by_artist_limit_50(token, artist_id,offset):
    '''Search "album" and "singles" given artist_id limit 50 per request at a time
        Return list of items '''
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = get_auth_header(token)

    querystring = {
                "include_groups": "album,single",
                "limit": 50,
                "offset": offset
                }

    result = requests.request("GET",url, headers= headers, params=querystring)
    
    json_result = result.json()
    items = json_result['items']
    return items

Test what `get_singles_albums_by_artist_limit_50()` return

In [33]:
#testing the first 50
album_single = get_singles_albums_by_artist_limit_50(token,artist['artist_id'],0)
album_single

[{'album_group': 'album',
  'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2kxP07DLgs4xlWz8YHlvfh'},
    'href': 'https://api.spotify.com/v1/artists/2kxP07DLgs4xlWz8YHlvfh',
    'id': '2kxP07DLgs4xlWz8YHlvfh',
    'name': 'NIKI',
    'type': 'artist',
    'uri': 'spotify:artist:2kxP07DLgs4xlWz8YHlvfh'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',
   'PE',
   'PH',
   'PL',
   'PT',
   'SG',
   'SK',
   'ES',
   'SE',
   'CH',
   'TW',
   'TR',
   'UY',
   'US',
   'GB',
   'AD',
   'LI',
   'MC',
   'ID',
   'JP',
   'TH',
   'VN',
   'RO',
   'IL',
   'ZA',
   'SA',
   'AE',
   'BH',
   'QA',

#### `get_all_singles_albums_by_artist()`

In [34]:
def get_all_singles_albums_by_artist(token, artist_id):
    '''Grabs all the singles and albums for the artist and return a dataframe
        with album_group, name, id, release_date, total_tracks, uri'''
    offset_counter = 0
    data = []
    while True:
        results = get_singles_albums_by_artist_limit_50(token,artist_id,offset_counter)
        offset_counter += 50
        data.extend(results)

        if len(results) != 50:
            break

    dataframe = pd.DataFrame(data, columns = ["album_group","name","id","release_date","total_tracks",'uri'])
    #drop duplicates
    dataframe = dataframe.drop_duplicates(subset=['name'], keep='first')
    #reset index
    dataframe = dataframe.reset_index(drop=True)
    #rename some columns for clariication
    rename_ls = {"name": "album_name", "id": "album_id", "release_date": "album_release_date" ,"uri": "album_uri"}
    dataframe = dataframe.rename(columns= rename_ls)

    #add artist_id to be able to tie it back to artist in relational query SQL
    id = np.full(shape = len(dataframe),fill_value = artist_id)
    dataframe['artist_id'] = id
    return dataframe


Test what `get_all_singles_albums_by_artist()` return

In [35]:
all_albums = get_all_singles_albums_by_artist(token,artist['artist_id'])
all_albums

,album_group,album_name,album_id,album_release_date,total_tracks,album_uri,artist_id
0,album,Nicole,5WR7ksPLp3kqFbDLTYpGfx,2022-08-12,12,spotify:album:5WR7ksPLp3kqFbDLTYpGfx,2kxP07DLgs4xlWz8YHlvfh
1,album,Shang-Chi and The Legend of The Ten Rings: The...,2kAqjStKcwlDD59H0llhGC,2021-09-03,18,spotify:album:2kAqjStKcwlDD59H0llhGC,2kxP07DLgs4xlWz8YHlvfh
2,album,MOONCHILD,59Y3t7Q4Vf8pw8STJ6fogW,2020-09-10,10,spotify:album:59Y3t7Q4Vf8pw8STJ6fogW,2kxP07DLgs4xlWz8YHlvfh
3,album,Zephyr,4E3FHEEdQkcuEd0D2GKRrX,2018-05-23,8,spotify:album:4E3FHEEdQkcuEd0D2GKRrX,2kxP07DLgs4xlWz8YHlvfh
4,single,Spotify Singles,5F6IRzJZGzLGb8R0GDMKSz,2022-09-14,2,spotify:album:5F6IRzJZGzLGb8R0GDMKSz,2kxP07DLgs4xlWz8YHlvfh
5,single,High School in Jakarta,34zREEtZQOmQbqFU4cIQuT,2022-08-05,1,spotify:album:34zREEtZQOmQbqFU4cIQuT,2kxP07DLgs4xlWz8YHlvfh
6,single,Oceans & Engines,2va673nk2JXgCxJeTiZdM9,2022-07-08,1,spotify:album:2va673nk2JXgCxJeTiZdM9,2kxP07DLgs4xlWz8YHlvfh
7,single,Before,6fXesrnlXstTyz9uurgYEY,2022-06-03,1,spotify:album:6fXesrnlXstTyz9uurgYEY,2kxP07DLgs4xlWz8YHlvfh
8,single,Split,7uAHGGN6s8u1BwkA3uKYiQ,2021-11-18,1,spotify:album:7uAHGGN6s8u1BwkA3uKYiQ,2kxP07DLgs4xlWz8YHlvfh
9,single,COMING HOME (feat. NIKI) [Long Distance Record...,3XTizmfvhOiFp5GMSTkRZ3,2021-10-01,2,spotify:album:3XTizmfvhOiFp5GMSTkRZ3,2kxP07DLgs4xlWz8YHlvfh


#### `get_album_single_popularity_by_id()`

In [36]:
def get_album_single_popularity_by_id(token, single_album_id):
    '''input one single/album id to search and return an popularity score'''

    url = f"https://api.spotify.com/v1/albums/{single_album_id}"
    headers = get_auth_header(token)

    result = requests.request("GET",url, headers= headers)
    
    json_result = result.json()
    popularity_score = json_result['popularity']
    return popularity_score
    

Test what `get_album_single_popularity_by_id()` return

In [37]:
#get back Popularity score of the album
get_album_single_popularity_by_id(token,"5MS3MvWHJ3lOZPLiMxzOU6")

92

#### `get_all_pop_for_album_single()`
Also renames the albums table columns to the ones that will be used later

In [38]:
def get_all_pop_for_album_single(token,albums_singles_df):
    '''input dataframe of all albums and singles of an artist and use the id column to get the popularity score return updated dataframe'''
    # get the number of rows in the dataframe
    n = len(albums_singles_df.index)
    #create a list of id
    id_list = albums_singles_df['album_id']
    #create an array of the same size as id to get the popularity score
    pop_score_list = np.empty(n)

    for i in range(len(id_list)):
        pop_score_list[i] = get_album_single_popularity_by_id(token,id_list[i])
    
    albums_singles_df['album_popularity'] = pop_score_list
    return albums_singles_df


Test what `get_all_pop_for_album_single()` return

In [39]:
#all albums and singles with popularity score
all_albums_pop = get_all_pop_for_album_single(token,all_albums)
all_albums_pop

,album_group,album_name,album_id,album_release_date,total_tracks,album_uri,artist_id,album_popularity
0,album,Nicole,5WR7ksPLp3kqFbDLTYpGfx,2022-08-12,12,spotify:album:5WR7ksPLp3kqFbDLTYpGfx,2kxP07DLgs4xlWz8YHlvfh,69.0
1,album,Shang-Chi and The Legend of The Ten Rings: The...,2kAqjStKcwlDD59H0llhGC,2021-09-03,18,spotify:album:2kAqjStKcwlDD59H0llhGC,2kxP07DLgs4xlWz8YHlvfh,68.0
2,album,MOONCHILD,59Y3t7Q4Vf8pw8STJ6fogW,2020-09-10,10,spotify:album:59Y3t7Q4Vf8pw8STJ6fogW,2kxP07DLgs4xlWz8YHlvfh,59.0
3,album,Zephyr,4E3FHEEdQkcuEd0D2GKRrX,2018-05-23,8,spotify:album:4E3FHEEdQkcuEd0D2GKRrX,2kxP07DLgs4xlWz8YHlvfh,59.0
4,single,Spotify Singles,5F6IRzJZGzLGb8R0GDMKSz,2022-09-14,2,spotify:album:5F6IRzJZGzLGb8R0GDMKSz,2kxP07DLgs4xlWz8YHlvfh,54.0
5,single,High School in Jakarta,34zREEtZQOmQbqFU4cIQuT,2022-08-05,1,spotify:album:34zREEtZQOmQbqFU4cIQuT,2kxP07DLgs4xlWz8YHlvfh,63.0
6,single,Oceans & Engines,2va673nk2JXgCxJeTiZdM9,2022-07-08,1,spotify:album:2va673nk2JXgCxJeTiZdM9,2kxP07DLgs4xlWz8YHlvfh,59.0
7,single,Before,6fXesrnlXstTyz9uurgYEY,2022-06-03,1,spotify:album:6fXesrnlXstTyz9uurgYEY,2kxP07DLgs4xlWz8YHlvfh,45.0
8,single,Split,7uAHGGN6s8u1BwkA3uKYiQ,2021-11-18,1,spotify:album:7uAHGGN6s8u1BwkA3uKYiQ,2kxP07DLgs4xlWz8YHlvfh,52.0
9,single,COMING HOME (feat. NIKI) [Long Distance Record...,3XTizmfvhOiFp5GMSTkRZ3,2021-10-01,2,spotify:album:3XTizmfvhOiFp5GMSTkRZ3,2kxP07DLgs4xlWz8YHlvfh,43.0


## Search for each track inside dataframe of all albums and singles

#### `get_tracks_by_album_limit_50()`

In [40]:
def get_tracks_by_album_limit_50(token, single_album_id,offset):
    '''input one single/album id to search and return first 50 tracks in the album'''

    url = f"https://api.spotify.com/v1/albums/{single_album_id}/tracks"
    headers = get_auth_header(token)

    querystring = {
                "limit": 50,
                "offset": offset
                }

    result = requests.request("GET",url, headers= headers, params=querystring)
    
    json_result = result.json()
    items = json_result['items']
    return items

Test what `get_tracks_by_album_limit_50` return

In [41]:
get_tracks_by_album_limit_50(token, "5MS3MvWHJ3lOZPLiMxzOU6",0)

[{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
    'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
    'id': '3TVXtAsR1Inumwj472S9r4',
    'name': 'Drake',
    'type': 'artist',
    'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/1URnnhqYAYcrqrcwql10ft'},
    'href': 'https://api.spotify.com/v1/artists/1URnnhqYAYcrqrcwql10ft',
    'id': '1URnnhqYAYcrqrcwql10ft',
    'name': '21 Savage',
    'type': 'artist',
    'uri': 'spotify:artist:1URnnhqYAYcrqrcwql10ft'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',


#### `get_all_tracks_for_one_album()`
This is mandatory to run even if the album has less than 50 track

In [42]:
def get_all_tracks_for_one_album(token, single_album_id):
    '''Grabs all the tracks of the album (function mainly useful if album has more than 50 tracks) and return dataframe with album_id attached'''
    offset_counter = 0
    data = []
    while True:
        results = get_tracks_by_album_limit_50(token,single_album_id,offset_counter)
        offset_counter += 50
        data.extend(results)

        if len(results) != 50:
            break
    
    ##dataframe = pd.DataFrame(data)
    #extract only relevant fields
    dataframe = pd.DataFrame(data, columns = ["name","id",'uri', "track_number"])
    dataframe = dataframe.reset_index(drop=True)

    #rename some columns for clariication
    rename_ls = {"name": "track_name", "id": "track_id","uri": "track_uri"}
    dataframe = dataframe.rename(columns= rename_ls)
    
    #include album id for easier identification later on
    album_id = np.full(shape = len(dataframe),fill_value = single_album_id)
    dataframe['album_id'] = album_id
    return dataframe


Test what `get_all_tracks_for_one_album()` return

In [43]:
get_all_tracks_for_one_album(token,"5MS3MvWHJ3lOZPLiMxzOU6")

,track_name,track_id,track_uri,track_number,album_id
0,Rich Flex,1bDbXMyjaUIooNwFE9wn0N,spotify:track:1bDbXMyjaUIooNwFE9wn0N,1,5MS3MvWHJ3lOZPLiMxzOU6
1,Major Distribution,46s57QULU02Voy0Kup6UEb,spotify:track:46s57QULU02Voy0Kup6UEb,2,5MS3MvWHJ3lOZPLiMxzOU6
2,On BS,34tz0eDhGuFErIuW3q4mPX,spotify:track:34tz0eDhGuFErIuW3q4mPX,3,5MS3MvWHJ3lOZPLiMxzOU6
3,BackOutsideBoyz,0wshkEEcJUQU33RSRBb5dv,spotify:track:0wshkEEcJUQU33RSRBb5dv,4,5MS3MvWHJ3lOZPLiMxzOU6
4,Privileged Rappers,7l2nxyx7IkBX5orhkALg0V,spotify:track:7l2nxyx7IkBX5orhkALg0V,5,5MS3MvWHJ3lOZPLiMxzOU6
5,Spin Bout U,2ZL7WZcjuYKi1KUDtp4kCC,spotify:track:2ZL7WZcjuYKi1KUDtp4kCC,6,5MS3MvWHJ3lOZPLiMxzOU6
6,Hours In Silence,0sSRLXxknVTQDStgU1NqpY,spotify:track:0sSRLXxknVTQDStgU1NqpY,7,5MS3MvWHJ3lOZPLiMxzOU6
7,Treacherous Twins,4Flfb4fGscN9kXPOduQLrv,spotify:track:4Flfb4fGscN9kXPOduQLrv,8,5MS3MvWHJ3lOZPLiMxzOU6
8,Circo Loco,7GeTsDIc5ykNB6lORO6Cee,spotify:track:7GeTsDIc5ykNB6lORO6Cee,9,5MS3MvWHJ3lOZPLiMxzOU6
9,Pussy & Millions (feat. Travis Scott),2KLwPaRDOB87XOYAT2fgxh,spotify:track:2KLwPaRDOB87XOYAT2fgxh,10,5MS3MvWHJ3lOZPLiMxzOU6


#### `get_all_tracks_from_albums()`

In [44]:
def get_all_tracks_from_albums(token, albums_singles_df):
    '''Input dataframe of album and singles extract album_id and run get_all_tracks_for_one_album() on for each album_id
        return dataframe of all tracks '''
    # Create empty dataframe to concatenate
    all_tracks = pd.DataFrame()
    album_ids = albums_singles_df['album_id']
    #loop through each album_id to get all the tracks within
    for i in album_ids:
        #the searched dataframe of one album
        temp = get_all_tracks_for_one_album(token,i)
        #concatenating onto the previous dataframe
        all_tracks = pd.concat([all_tracks,temp], axis = 0)
    all_tracks = all_tracks.reset_index(drop=True)
    return all_tracks

Test what `get_all_tracks_from_albums()` return

In [45]:
all_tracks_df = get_all_tracks_from_albums(token,all_albums_pop)
all_tracks_df

,track_name,track_id,track_uri,track_number,album_id
0,Before,2JcLhDcdmV8ZZ7aTvyIp6s,spotify:track:2JcLhDcdmV8ZZ7aTvyIp6s,1,5WR7ksPLp3kqFbDLTYpGfx
1,High School in Jakarta,1zxfRSZcaonV1VXcY0PgY5,spotify:track:1zxfRSZcaonV1VXcY0PgY5,2,5WR7ksPLp3kqFbDLTYpGfx
2,Backburner,4kkWvBCT6wq5NHoJjYRaPU,spotify:track:4kkWvBCT6wq5NHoJjYRaPU,3,5WR7ksPLp3kqFbDLTYpGfx
3,Keeping Tabs,2zz4plJxtgMIsQg5SMl2Bg,spotify:track:2zz4plJxtgMIsQg5SMl2Bg,4,5WR7ksPLp3kqFbDLTYpGfx
4,The Apartment We Won't Share,7uvtPp6So6MtrGSC3LpKfs,spotify:track:7uvtPp6So6MtrGSC3LpKfs,5,5WR7ksPLp3kqFbDLTYpGfx
...,...,...,...,...,...
81,Dancing with the Devil,1r5DB5UsDMtCifQxvjpbZL,spotify:track:1r5DB5UsDMtCifQxvjpbZL,1,4SXjVeDcEp8583Sijc0EIc
82,Vintage,0FmZURyRtan712fkO8JNr6,spotify:track:0FmZURyRtan712fkO8JNr6,1,2JgZ5yio1GjbZGUVl7BwmS
83,Chilly,6H3TW6uEe3RxW8CcnXJfq2,spotify:track:6H3TW6uEe3RxW8CcnXJfq2,1,5NmHTG1AIpZ9yjwzih2Kni
84,I Like U,6f7BE3VcZScJx6n3wsf5Z3,spotify:track:6f7BE3VcZScJx6n3wsf5Z3,1,4Hnz0SvIaKVJkuwe93fVy6


## Search for each tracks' popularity score from dataframe of all tracks

#### `get_track_popularity_by_id()`

In [46]:
def get_track_popularity_by_id(token, track_id):
    '''Search the track_id and get their popularity score'''
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    headers = get_auth_header(token)

    result = requests.request("GET",url, headers= headers)
    
    json_result = result.json()
    popularity_score = json_result['popularity']
    return popularity_score

Test what `get_track_popularity_by_id()` return

In [47]:
get_track_popularity_by_id(token,"1bDbXMyjaUIooNwFE9wn0N")

89

#### `get_pop_for_all_tracks()`

In [48]:
def get_pop_for_all_tracks(token,all_tracks_df):
    '''input dataframe of all tracks and use the id column to get the popularity score return updated dataframe'''
    # get the number of rows in the dataframe
    n = len(all_tracks_df.index)
    #create a list of id
    id_list = all_tracks_df['track_id']
    #create an array of the same size as id to get the popularity score
    pop_score_list = np.empty(n)

    for i in range(len(id_list)):
        pop_score_list[i] = get_track_popularity_by_id(token,id_list[i])
    
    all_tracks_df['track_popularity'] = pop_score_list
    return all_tracks_df

In [49]:
get_pop_for_all_tracks(token,all_tracks_df)

,track_name,track_id,track_uri,track_number,album_id,track_popularity
0,Before,2JcLhDcdmV8ZZ7aTvyIp6s,spotify:track:2JcLhDcdmV8ZZ7aTvyIp6s,1,5WR7ksPLp3kqFbDLTYpGfx,53.0
1,High School in Jakarta,1zxfRSZcaonV1VXcY0PgY5,spotify:track:1zxfRSZcaonV1VXcY0PgY5,2,5WR7ksPLp3kqFbDLTYpGfx,67.0
2,Backburner,4kkWvBCT6wq5NHoJjYRaPU,spotify:track:4kkWvBCT6wq5NHoJjYRaPU,3,5WR7ksPLp3kqFbDLTYpGfx,59.0
3,Keeping Tabs,2zz4plJxtgMIsQg5SMl2Bg,spotify:track:2zz4plJxtgMIsQg5SMl2Bg,4,5WR7ksPLp3kqFbDLTYpGfx,57.0
4,The Apartment We Won't Share,7uvtPp6So6MtrGSC3LpKfs,spotify:track:7uvtPp6So6MtrGSC3LpKfs,5,5WR7ksPLp3kqFbDLTYpGfx,48.0
...,...,...,...,...,...,...
81,Dancing with the Devil,1r5DB5UsDMtCifQxvjpbZL,spotify:track:1r5DB5UsDMtCifQxvjpbZL,1,4SXjVeDcEp8583Sijc0EIc,15.0
82,Vintage,0FmZURyRtan712fkO8JNr6,spotify:track:0FmZURyRtan712fkO8JNr6,1,2JgZ5yio1GjbZGUVl7BwmS,35.0
83,Chilly,6H3TW6uEe3RxW8CcnXJfq2,spotify:track:6H3TW6uEe3RxW8CcnXJfq2,1,5NmHTG1AIpZ9yjwzih2Kni,53.0
84,I Like U,6f7BE3VcZScJx6n3wsf5Z3,spotify:track:6f7BE3VcZScJx6n3wsf5Z3,1,4Hnz0SvIaKVJkuwe93fVy6,66.0


***

### Additional functions for ease of use

#### `get_album_id()`

In [50]:
def get_album_id(albums_singles_df):
    '''Gets the array of album_ids from a dataframe'''
    album_ids = albums_singles_df['album_id'].unique()
    return album_ids

In [51]:
get_album_id(all_albums)

array(['5WR7ksPLp3kqFbDLTYpGfx', '2kAqjStKcwlDD59H0llhGC',
       '59Y3t7Q4Vf8pw8STJ6fogW', '4E3FHEEdQkcuEd0D2GKRrX',
       '5F6IRzJZGzLGb8R0GDMKSz', '34zREEtZQOmQbqFU4cIQuT',
       '2va673nk2JXgCxJeTiZdM9', '6fXesrnlXstTyz9uurgYEY',
       '7uAHGGN6s8u1BwkA3uKYiQ', '3XTizmfvhOiFp5GMSTkRZ3',
       '1BLMrCc9WNdafvEVXxdOab', '2HPj0XZe9WduSsyKTQqgVa',
       '2gg6B0XKlAW0XwIMIrwk2n', '6LAcWcel05QWZz1dLGAbk7',
       '2gCRiVJ8w6f64f2rtb3TAD', '2SBsj6Pk4ZCY5lmsWDjjvi',
       '5GW3Hm4C7AkrClwby88sFJ', '098kaaaRAjWOrvoLRQHU2i',
       '3Jbl4tevdetpELBgXwBjKp', '1KdgJeKOFZ4fZteMqji9Rj',
       '3Y3CwwjIi03RRyJohYzKXh', '00ikjZ4maHRaqxMI96NPQD',
       '3HLwiAL4LbHVwQaVCl3tnR', '6a1p03zluxNwXTVdm9IStw',
       '3xUR1JT0b2uC1SZ54flBll', '4SXjVeDcEp8583Sijc0EIc',
       '2JgZ5yio1GjbZGUVl7BwmS', '5NmHTG1AIpZ9yjwzih2Kni',
       '4Hnz0SvIaKVJkuwe93fVy6', '3ZqxhYWOhGEXYrYUkuflL0'], dtype=object)